# Projeto #2 - Classificador supervisionado

Antes de começar, leia as [Instruções](https://github.com/thvmm/pos-ds-ia/tree/master/projeto_2#instru%C3%A7%C3%B5es) e os [Critérios de Avaliação](https://github.com/thvmm/pos-ds-ia/tree/master/projeto_2#crit%C3%A9rios-de-avalia%C3%A7%C3%A3o)


### 1) Qual a base escolhida?



In [77]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)

#leitura do arquivo csv
data = pd.read_csv('covtype.csv')
data.head()


,Elevation,Aspect,slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Wilderness_Area_4,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_1.1,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


### 2) **(10%)** Pré-processamento: entendimento do conjunto de dados
- Quais são minhas features?
- Quais são minhas classes?
- Como estão distribuidas minhas classes?
- Checagem se os valores estão dentro de um limite permitido ou razoável.
- Tratamento de valores ausentes por eliminação ou substituição.
- Conversão do tipo de dados.


In [78]:
print('Dados:')
print('Número de Registros:', data.shape[0])
print('Número de Featues:', data.shape[1])

Dados:
Número de Registros: 581012
Número de Featues: 55


In [79]:
data.dtypes

Elevation                             int64
Aspect                                int64
slope                                 int64
Horizontal_Distance_To_Hydrology      int64
Vertical_Distance_To_Hydrology        int64
Horizontal_Distance_To_Roadways       int64
Hillshade_9am                         int64
Hillshade_Noon                        int64
Hillshade_3pm                         int64
Horizontal_Distance_To_Fire_Points    int64
Wilderness_Area_1                     int64
Wilderness_Area_2                     int64
Wilderness_Area_3                     int64
Wilderness_Area_4                     int64
Soil_Type_1                           int64
Soil_Type_2                           int64
Soil_Type_3                           int64
Soil_Type_4                           int64
Soil_Type_5                           int64
Soil_Type_6                           int64
Soil_Type_7                           int64
Soil_Type_8                           int64
Soil_Type_9                     

In [44]:
#Quais são as minhas features? (covtype.info)

#Elevation                               Elevation in meters
#Aspect                                  Aspect in degrees azimuth
#Slope                                   Slope in degrees
#Horizontal_Distance_To_Hydrology        Horz Dist to nearest surface water features
#Vertical_Distance_To_Hydrology          Vert Dist to nearest surface water features
#Horizontal_Distance_To_Roadways         Horz Dist to nearest roadway
#Hillshade_9am                           Hillshade index at 9am, summer solstice
#Hillshade_Noon                          Hillshade index at noon, summer soltice
#Hillshade_3pm                           Hillshade index at 3pm, summer solstice
#Horizontal_Distance_To_Fire_Points      Horz Dist to nearest wildfire ignition points
#Wilderness_Area (4 binary columns)      Wilderness area designation
#Soil_Type (40 binary columns)           Soil Type designation
#Cover_Type (7 types)                    Forest Cover Type designation

In [45]:
#Quais são minhas classes? (covtype.info)
#classe                     qtde registros
#Spruce-Fir:                211840 
#Lodgepole Pine:            283301 
#Ponderosa Pine:             35754 
#Cottonwood/Willow:           2747 
#Aspen:                       9493 
#Douglas-fir:                17367 
#Krummholz:                  20510   

In [80]:
#Como estão distribuidas minhas classes?
data.groupby('Cover_Type').size()

Cover_Type
1    211840
2    283301
3     35754
4      2747
5      9493
6     17367
7     20510
dtype: int64

In [81]:
#Checagem se os valores estão dentro de um limite permitido ou razoável.
data.describe()

,Elevation,Aspect,slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_1,Wilderness_Area_2,Wilderness_Area_3,Wilderness_Area_4,Soil_Type_1,Soil_Type_2,Soil_Type_3,Soil_Type_4,Soil_Type_5,Soil_Type_6,Soil_Type_7,Soil_Type_8,Soil_Type_9,Soil_Type_10,Soil_Type_11,Soil_Type_12,Soil_Type_13,Soil_Type_14,Soil_Type_15,Soil_Type_16,Soil_Type_17,Soil_Type_18,Soil_Type_1.1,Soil_Type_20,Soil_Type_21,Soil_Type_22,Soil_Type_23,Soil_Type_24,Soil_Type_25,Soil_Type_26,Soil_Type_27,Soil_Type_28,Soil_Type_29,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,0.448865,0.051434,0.436074,0.063627,0.005217,0.012952,0.008301,0.021335,0.002749,0.011316,0.000181,0.000308,0.001974,0.056168,0.021359,0.051584,0.030001,0.001031,0.000005,0.004897,0.005890,0.003268,0.006921,0.015936,0.001442,0.057439,0.099399,0.036622,0.000816,0.004456,0.001869,0.001628,0.198356,0.051927,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,0.497379,0.220882,0.495897,0.244087,0.072039,0.113066,0.090731,0.144499,0.052356,0.105775,0.013442,0.017550,0.044387,0.230245,0.144579,0.221186,0.170590,0.032092,0.002272,0.069804,0.076518,0.057077,0.082902,0.125228,0.037950,0.232681,0.299197,0.187833,0.028551,0.066605,0.043193,0.040318,0.398762,0.221879,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [61]:
#Tratamento de valores ausentes por eliminação ou substituição.
data.isnull().sum()
#sem valores ausentes

Elevation                             0
Aspect                                0
slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area_1                     0
Wilderness_Area_2                     0
Wilderness_Area_3                     0
Wilderness_Area_4                     0
Soil_Type_1                           0
Soil_Type_2                           0
Soil_Type_3                           0
Soil_Type_4                           0
Soil_Type_5                           0
Soil_Type_6                           0
Soil_Type_7                           0
Soil_Type_8                           0
Soil_Type_9                           0
Soil_Type_10                          0
Soil_Type_11                          0


### 3) **(80%)** Nos blocos seguintes implemente seus classificadores (serão implementados 2 métodos diferentes).

#### 3.1) Qual método escolhido?

Naive Bayes

O algoritmo “Naive Bayes” é um classificador probabilístico baseado no “Teorema de Bayes”, o qual foi criado por Thomas Bayes (1701 - 1761) 

#### 3.2) **(10%)** Baseline - Implemente seu classificador da forma mais simples possível para esse ser seu baseline

In [96]:
# Implementação. Use mais blocos se achar que ficará mais organizado.
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
dataNB = data
y  = dataNB['Cover_Type']
dataNB = dataNB.drop(['Cover_Type'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(dataNB, y, test_size = 0.35)

# listagem dos registros de treino e teste
print("{} amostras de treino.".format(X_train.shape[0]))
print("{} amostras de teste.".format(X_test.shape[0]))
print(y_train.value_counts())


377657 amostras de treino.
203355 amostras de teste.
2    184038
1    137751
3     23309
7     13202
6     11316
5      6249
4      1792
Name: Cover_Type, dtype: int64


In [83]:
clfNB = GaussianNB()
clfNB.fit(X_train, y_train)

y_pred = classificador.predict(X_test)
ACC = accuracy_score(y_pred, y_test) * 100

print("Acurácia de: " + str(ACC) + "% com GaussianNB")

Acurácia de: 45.70480194733348% com GaussianNB


In [84]:
#Testes com Algoritmo BernoulliNB   
ClfBNB = BernoulliNB()
ClfBNB.fit(X_train, y_train)
y_prd = ClfBNB.predict(X_test)
ACC = accuracy_score(y_prd, y_test) * 100
print("Acuracia de " + str(ACC) + "% com BernoulliNB")

Acuracia de 62.78134297165057% com BernoulliNB


Algoritmo BernoulliNB está com acurácia maior do que o GaussianNB... 
preciso avaliar melhor!

In [85]:

#eliminar features que estão com valores menor que 0.05. por padrão o VarianceThreshold elimina valores = 0.
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.05) #Manter todos os dados com variação maior do que 0.05!
reduce_data = sel.fit_transform(dataNB)
print(reduce_data.shape) #Registros / Qtde de features...

(581012, 33)


In [90]:
X_train, X_test, y_train, y_test = train_test_split(reduce_data,y,test_size=0.35)

ClfBNB.fit(X_train, y_train)

y_pred = ClfBNB.predict(X_test)
ACC = accuracy_score(y_pred, y_test) * 100
print("Acuracia apos ajustes: " + str(ACC) + "%" )
#diminuiu acurácia.... vamos avaliar

Acuracia apos ajustes: 0.6202404661798333%


In [93]:
# novos testes para avaliar acuracia
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.01) #Manter todos os dados com variação maior do que 0.01!
reduce_data = sel.fit_transform(dataNB)
print(reduce_data.shape) #Quantidade de faatures aumentou em relacao ao anterior
#Melhor ou pior??

(581012, 33)


In [94]:
X_train, X_test, y_train, y_test = train_test_split(df_variance_threshold,y,test_size=0.35)

ClfBNB.fit(X_train, y_train)

y_pred = ClfBNB.predict(X_test)
acuracia = accuracy_score(y_pred, y_test)
print("Acuracia apos ajustes: " + str(acuracia))


Acuracia apos ajustes: 0.6264561972904527


In [ ]:
#Mesmo com tunning acurácia ficou dentro dos 62%...

#### 3.4) **(10%)** Tunning - Agora que temos um resultado promissor, vamos tentar melhorar o resultado alterando um ou mais hiper-parametro. Compare os resultados.

#### 3.5) Qual método escolhido?

Indique o método escolhido

Random Forest

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

#### 3.6) **(10%)** Baseline - Implemente seu classificador da forma mais simples possível para esse ser seu baseline

In [105]:
from sklearn.ensemble import RandomForestClassifier
dataRFC = data
y  = dataRFC['Cover_Type']
dataNB = dataRFC.drop(['Cover_Type'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(dataRFC, y,  test_size=0.35)
#y = cover_type

classifierRandomForest = RandomForestClassifier(max_depth=2, random_state=0)
classifierRandomForest.fit(X_train, y_train)


y_pred = classifierRandomForest.predict(X_test)

ACC = accuracy_score(y_pred, y_test) * 100

print("Acurácia de : " + str(ACC) + "%")

Acurácia de : 84.82536258490913%


#### 3.7) **(20%)** Versão 1 - O que podemos fazer para melhorar nosso baseline? Aplique técnicas como redução de dimensionalidade, normalização ou outras. Compare os resultados.

#### 3.8) **(10%)** Tunning - Agora que temos um resultado promissor, vamos tentar melhorar o resultado alterando um ou mais hiper-parametro. Compare os resultados.

In [ ]:
# Implementação. Use mais blocos se achar que ficará mais organizado.

Algoritmo BernoulliNB teve um resultado promissor, porém a Random Forest obtém um resultado melhor!
 